<div align="center" class="alert alert-info" role="alert">
    <h1>
        Sibur Challenge 2020
        <br>
        <small class="text-muted">онлайн-чемпионат по анализу данных</small>
    </h1>
</div>
<div align="right" class="alert alert-info" role="alert">
    <h3>Евгения Хомякова (в составе команды "+-3сигмы"), 2020/12</h3>
</div>

## Сопоставление названий

При поиске новых клиентов СИБУРу приходится обрабатывать информацию о миллионах новых компаний из различных источников. Названия компаний при этом могут иметь разное написание, содержать сокращения или ошибки, быть аффилированными с компаниями, уже известными СИБУРу.

Для более эффективной обработки информации о потенциальных клиентах, СИБУРу необходимо знать, связаны ли два названия (т.е. принадлежат одной компании или аффилированным компаниям).

В этом случае СИБУР сможет использовать уже известную информацию о самой компании или об аффилированных компаниях, не дублировать обращения в компанию или не тратить время на нерелевантные компании или дочерние компании конкурентов.

Тренировочная выборка содержит пары названий из разных источников (в том числе, пользовательских) и разметку.

Разметка получена частично вручную, частично - алгоритмически. Кроме того, разметка может содержать ошибки. Вам предстоит построить бинарную модель, предсказывающую, являются ли два названия связанными. Метрика, используемая в данной задаче - F1.

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

import re

from tqdm import tqdm
import pycountry

from transliterate import translit

import dask.bag as db
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

import textdistance as td

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Загрузка данных

In [2]:
train = pd.read_csv("train.csv", index_col="pair_id")
test = pd.read_csv("test.csv", index_col="pair_id")

In [3]:
train_raw = pd.read_csv("train.csv", index_col="pair_id")
test_raw = pd.read_csv("test.csv", index_col="pair_id")

In [4]:
train.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497819 entries, 1 to 497819
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name_1        497819 non-null  object
 1   name_2        497819 non-null  object
 2   is_duplicate  497819 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 85.0 MB


In [6]:
test.head()

,name_1,name_2
pair_id,,
1,Sun Blinds Decoration Inc.,Indl De Cuautitlan Sa Cv
2,Eih Ltd.,"Dongguan Wei Shi Plastic Product Co., Ltd."
3,Jsh Ltd. (Hk),Arab Shipbuilding And Repair Yard C
4,Better Industrial Ltd.,Farmacap Industria E Comercio Ltda
5,Equipos Inoxidables Del Norte Sa De Cv,Bel Inc.


In [7]:
test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213249 entries, 1 to 213249
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name_1  213249 non-null  object
 1   name_2  213249 non-null  object
dtypes: object(2)
memory usage: 34.8 MB


# Статистика таргета

In [8]:
train.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [9]:
train[train.is_duplicate==1].sample(10)

,name_1,name_2,is_duplicate
pair_id,,,
121197,Bridgestone Firestone Do Brasil,Bridgestone International Group,1
432422,Soprema Iberia,Soprema Iberia S.L.,1
16641,Bridgestone Ncr,Bridgestone (China) Research And Development C...,1
116906,"Michelin Siam Co., Ltd.",Sociedade Michelin De Participacoes Indust E C...,1
36011,Michelin India Technology Center Llp,"Michelin Siam Co., Ltd.",1
118899,Bridgestone Firestone De Mexico Sa De Cv,Bridgestone Ncr,1
441150,Bridgestone Firestone Venezolana C,Bridgestone Tire Co.,1
36565,Exxonmobil Chemical Americas 22777 Springwoods...,Exxonmobil Chemical Americas On,1
176621,Trinseo Europe Gmb H Trade Register 20162359 T...,Trinseo Europe Gmb H Trade Register 20162359,1


In [10]:
train[train.is_duplicate==0].sample(10)

,name_1,name_2,is_duplicate
pair_id,,,
224453,Sant Rubbers Ltd.,"St.Marys Rubbers Pvt., Ltd.",0
318777,BAL TABAN,BITOUMINA SA,0
445537,A S International,All Ways Forwarding International Inc.,0
299487,Sheikh Imp. & Exp.,Xiamen Gulong Imp. & Exp.,0
474144,Al Amal Co.,Anti Malaria Campaign,0
439498,Cashbasisenterprises Llc,Eterna S.A.,0
12595,J & K Industries,Sun Pharmaceutical Industries Ltd.,0
17420,Nad International S A De C V,Global Agri Trade Corp.,0
68961,"Kunshan Daou New Materials Co., Ltd.",Mactex Corporation,0


In [11]:
# убираем неодинарные пробелы и пробелы в начале и конце строк
def clean_spaces():
    for df in [train, test]:
        for name in ['name_1', 'name_2']:
            df[name] = df[name].str.replace('\s+', ' ', regex=True)
            df[name] = df[name].str.strip()
            df[name] = df[name].str.lstrip()

# Очистка данных

### Убираем аномалии

In [12]:
# нижний регистр
for df in [train, test]:
    for name in ['name_1', 'name_2']:
        df[name] = df[name].str.lower()

In [13]:
train.duplicated().sum()

42

In [14]:
test.duplicated().sum()

15

In [15]:
train = train.drop_duplicates()

In [16]:
train.sample(15)

,name_1,name_2,is_duplicate
pair_id,,,
201595,primeur ltd.,pt lawe adyaprima,0
183137,co. one,azam enterprises,0
58917,n j international inc.,c & t logistics agente de carga e transporte ltda,0
107321,infinite cr strips industries ltd.,aspire industries,0
297758,sam international,tropic seafood,0
153231,black rose industries ltd.,indian cork industries,0
189407,radix usa co.,rankin usa inc.,0
63027,a.l.,mach 1 global services,0
423770,ilg,mercantil s a,0


In [17]:
anomalies = [
    # убрать китайское и арабское
    '[\u4E00-\u9FFF]+|[\u0621-\u064A]+|[\u0627-\u064a]+',
    # биржевые идентификаторы
    '\(?tse:\d+\)?', '\(?nyse:.+\)?', '\(?nasdaq.+\)?', '\(?bse:\d+\)?',
    #
    'tax\s?id\s?\d+',
    # убрать кавычки, апострофы, запятые, звездочки, скобки
    ',',
    '/',
    '\.',
    '`','`s',
    '\*',
    '\+',
    '\[', '\]',
    '\(.*\)',
    '#',
    ':',
    '"',
    '\?',
    "'s",
    "'",
    '-',
    '«', '»',
    '&',
]

In [18]:
%%time
for a in tqdm(anomalies):
    for df in [train, test]:
        df['name_1'] = df['name_1'].str.replace(a, ' ', n=-1, regex=True)
        df['name_2'] = df['name_2'].str.replace(a, ' ', n=-1, regex=True)
clean_spaces()

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:28<00:00,  1.10s/it]


Wall time: 33.2 s


In [19]:
# опечатки некоторые исправим
for df in [train, test]:
    for series_name in ['name_1', 'name_2']:
        df[series_name] = df[series_name].str.replace(r'g\s?m\s?b\s?h', 'gmbh', regex=True)
        df[series_name] = df[series_name].str.replace('mexico', ' ', regex=True) #очень много где есть, мешает чистке

Сложный случай с ё: в трейне она одна, но заменим везде "ё" ее на "о":

In [20]:
train[train.name_1.str.contains('ё') | train.name_2.str.contains('ё')]

,name_1,name_2,is_duplicate
pair_id,,,
24723,dorken gmbh co kg,ооо дёркен,1


А также другие символы с точечками:

In [21]:
dots_replace_dict = {
    'ё': 'о',
    "é": "e",
    "ę": "e",
    "è": "e",
    
    "í": "i",
    
    "ú": "u",
    "ü": "u",
    "ű": "u",
    
    "ö": "o",
    "ó": "o",
    "õ": "o",
    "ő": "o",
    
    "ä": "a",
    "á": "a",
    "ä": "a",
    "ã": "a", 
    "ç": "c",
    "ł": "l",
    
    "ñ": "n",
    "ş": "s"
}

In [22]:
for key, value in tqdm(dots_replace_dict.items()):
    for df in [train, test]:
        df['name_1'] = df['name_1'].str.replace(key, value, regex=True)
        df['name_2'] = df['name_2'].str.replace(key, value, regex=True)
clean_spaces()

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:15<00:00,  1.27it/s]


In [23]:
train.loc[24723]

name_1          dorken gmbh co kg
name_2                 ооо доркен
is_duplicate                    1
Name: 24723, dtype: object

In [24]:
train.duplicated().sum()

3586

In [25]:
train = train.drop_duplicates()

## Проверка полных дубликатов 1

In [32]:
train['full_duplicate'] = train.apply(lambda x: set(x.name_1.split()) == set(x.name_2.split()), axis=1).astype('int')

In [33]:
train[(train.full_duplicate == 1) & (train.is_duplicate == 0)].shape

(16, 4)

In [29]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate == 1, 'name_1'] = train.loc[train.full_duplicate == 1, 'name_2'].values

In [34]:
# проверим полные дубликаты в тесте
test['full_duplicate'] = test.apply(lambda x: set(x.name_1.split()) == set(x.name_2.split()), axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate == 1, 'name_1'] = test.loc[test.full_duplicate == 1, 'name_2'].values

Отсечем от первого имени одно слово с конца:

In [35]:
def del1_word_n1(row):
    n1 = row.name_1.split()
    n2 = row.name_2.split()
    if len(n1) > 1:
        return set(n1[:-1]) == set(n2)
    else:
        return False

In [36]:
train['full_duplicate_1'] = train.apply(del1_word_n1, axis=1).astype('int')

In [38]:
train[(train.full_duplicate_1 == 1) & (train.is_duplicate == 0)].shape

(34, 5)

In [39]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate_1 == 1, 'name_1'] = train.loc[train.full_duplicate_1 == 1, 'name_2'].values

In [40]:
# отметим такие же дубликаты в тесте
test['full_duplicate_1'] = test.apply(del1_word_n1, axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate_1 == 1, 'name_1'] = test.loc[test.full_duplicate_1 == 1, 'name_2'].values

In [41]:
test[test['full_duplicate_1'] == 1].shape

(31, 4)

Класс!

То же самое, но от второго названия уберем одно слово:

In [42]:
def del1_word_n2(row):
    n1 = row.name_1.split()
    n2 = row.name_2.split()
    if len(n1) > 1:
        return set(n1) == set(n2[:-1])
    else:
        return False

In [43]:
train['full_duplicate_2'] = train.apply(del1_word_n2, axis=1).astype('int')

In [44]:
train[(train.full_duplicate_2 == 1) & (train.is_duplicate == 0)].shape

(24, 6)

In [45]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate_2 == 1, 'name_2'] = train.loc[train.full_duplicate_2 == 1, 'name_1'].values

In [46]:
# удалим эти столбцы
train['label'] = train[['full_duplicate','full_duplicate_1','full_duplicate_2']].max(axis=1)
train = train.drop(['full_duplicate', 'full_duplicate_1', 'full_duplicate_2'], axis=1)

In [48]:
# отметим такие же дубликаты в тесте
test['full_duplicate_2'] = test.apply(del1_word_n2, axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate_2 == 1, 'name_2'] = test.loc[test.full_duplicate_2 == 1, 'name_1'].values

In [49]:
# в тесте сделаем столбец разметки на основе сравнения
test['label'] = test[['full_duplicate','full_duplicate_1','full_duplicate_2']].max(axis=1)
test = test.drop(['full_duplicate', 'full_duplicate_1', 'full_duplicate_2'], axis=1)

In [47]:
train[(train.label == 1) & (train.is_duplicate == 0)].shape

(74, 4)

In [50]:
test[test.label==1].shape

(104, 3)

### Продолжаем чистку

Некоторые названия выглядят как 'буква( буква) буква':

In [51]:
reg = re.compile('^(\w)\s(\w\s)')
train[(train.name_1.str.contains(reg) | train.name_2.str.contains(reg))]

,name_1,name_2,is_duplicate,label
pair_id,,,,
10,beijing zhongyi rongda tech trading co ltd,n j international inc,0,0
11,r i intl,rass mfg india pvt ltd,0,0
15,warehouse jonesboro,s k international,0,0
25,h r international,a r,0,0
26,s r international,cala trading group s de r l de c v,0,0
...,...,...,...,...
497761,s r international,peliculas utiles sa de cv,0,0
497796,mitsui chemicals india private ltd,h r international,0,0
497797,oec freight inc,s r international,0,0


Объединим их:

In [52]:
for df in [train, test]:
    df['name_1'] = df['name_1'].str.replace('^(\w)\s(\w)\s(\w\s?)', '\\1\\2\\3', regex=True)
    df['name_2'] = df['name_2'].str.replace('^(\w)\s(\w)\s(\w\s?)', '\\1\\2\\3', regex=True)

    df['name_1'] = df['name_1'].str.replace('^(\w)\s(\w\s?)', '\\1\\2', regex=True)
    df['name_2'] = df['name_2'].str.replace('^(\w)\s(\w\s?)', '\\1\\2', regex=True)

In [53]:
train.duplicated().sum()

126

In [54]:
train = train.drop_duplicates()

## Удаление legal entities

https://en.wikipedia.org/wiki/List_of_legal_entity_types_by_country

https://en.wikipedia.org/wiki/Private_limited_company

In [55]:
enity_list = [
    # все в этом словаре определено исключительно на трейне
    'san\s?v?e?\s?tic', # +
    'co',
    #private limited company => ltd in UK and Commonwealth
    'private ltd',
    'private limited',
    'pvt\s?ltd',
    'pvt',
    'gmbh', #germany
    'kft', #hungary
    'pty\s?l?t?d?', #australia
    'l\s?l\s?c', #usa
    'sp\s?z\s?o\s?o', #poland
    'pte\s?l?t?d?', #singapore
    'lda', #portugal
    'pt', #indonesia
    'p\s?m\s?e', 'w\s?l\s?l',
    'ооо', #russia
    'общество\sс\sогр\S*\sответ\S*',
    's\s?r\s?o', #slovakia/ceska_republika
    'd\s?o\s?o', #west_europe
    's\s?i\s?a', #latvia
    '(e\s?i\s?r)?\s?l\s?t\s?d\s?a', #south_america
    'e\s?i\s?r\s?l', #south_america, #eirl
    's\s?[pac]\s?r\s?l', #belgium, france/africa
    's\s?a\s?s', #sas
    'a\s?s', #turkey
    '(rls)?\s?s?\s?de\s?r\s?l\s?(de)?\s?c\s?v', #mexico (rls) (s) de rl de cv
    's?\s?de\s?r\s?l', #mexico
    '(de)?\s?s?s\s?a de cv', #mexico
    's\s?a\s?p\s?i\s?de\s?c\s?v', #mexico
    
    #public limited company => plc in UK and Commonwealth
    'public ltd',
    'p\s?l\s?c', #uk
    's\s?a\d?\s?de\s?c\s?v', #mexico + 
    's\s?a',
    'sociedad', 'anonima',
    '[зо]?ао', #russia
    'ag', #germany
    's\s?p\s?a', #italy
    
    # если после всего этого останется ltd еще
    'l\s?t\s?d',
    'limited',
    
    'sti', 'l\s?p', 's\s?l\s?u', 'c\s?a', 'b\s?v', 'n\s?v',  'sp\s?k?',
    '(de)?\s?c\s?v',
]

In [56]:
%%time
for le in tqdm(enity_list):
    le_reg = re.compile('(^|\s+)'+le+'(\s+|$)')
    for df in [train, test]:
        df['name_1'] = df['name_1'].str.replace(le_reg, ' ', regex=True)
        df['name_2'] = df['name_2'].str.replace(le_reg, ' ', regex=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [02:16<00:00,  2.79s/it]

Wall time: 2min 16s


In [57]:
clean_spaces()

In [58]:
train.sample(5)

,name_1,name_2,is_duplicate,label
pair_id,,,,
391449,diamond rubber products,rohit rubber corporation,0,0
477401,sto corp,jsw steel,0,0
478177,forte,huicheng foreign processing assembling service,0,0
398654,pro rubber,polyhose india,0,0
476310,xiamen gulong imp exp,lucky imp exp,0,0


In [59]:
train.duplicated().sum()

6554

In [60]:
train = train.drop_duplicates()

## Стоп-слова

In [61]:
stop_words = [
    'the', 'of', 'do', 'and', "to", 'e', 've', 'de',
    'automotive',
    'bank', 
    'chemicals?',
    'company','corporation', 'corp', 'co', 'comerci\S*', "com", 'city',
    'distribution', 
    'equipment', 'exp', 'enterprise\S*', 'electronic\S*','engineering', 
    'global', 'general', 'group',
    'imp', 'importadora', 'international', 'industr\S*', 'inds', 'inc',
    'kg',
    'logistic\S*', 'lojistik',
    'mfg','material\S*',
    'plastic\S*', 'products','polymer\S*',
    'rubber', 'ram',
    "supply", "systems", 'solutions', 'sports', 'service\S*', "synthetic", 'sociedad', 'shoes',
    'textile', 'trad\S*', 'technolog\S*', 'tech',  "transport\S*", 't[iy]re',
    "united", 
    '\w', #любая одиночная буква
    'компания', 'филиал\sкомпании', 'филиал', 'снг', 'рус',
]

In [62]:
for stop_word in tqdm(stop_words):
    stop_word_reg = re.compile('(^|\s+)'+stop_word+'(\s+|$)')
    for df in [train, test]:
        for name in ['name_1', 'name_2']:
            df[name] = df[name].str.replace(stop_word_reg, ' ', regex=True)
clean_spaces()

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [02:31<00:00,  2.37s/it]


In [63]:
train.duplicated().sum()

9584

In [64]:
train = train.drop_duplicates()

### Транслитерация русского

In [65]:
russian = re.compile(r'[А-Яа-я]+')

In [66]:
(train.name_1.str.contains(russian) | train.name_2.str.contains(russian)).sum()

1829

In [67]:
train[train.name_1.str.contains(russian) | train.name_2.str.contains(russian)].sample(5)

,name_1,name_2,is_duplicate,label
pair_id,,,,
263705,soprema polska,химинвест нпф,0,0
117913,garland,химинвест групп,0,0
57839,goodyear,полимаркет,0,0
97152,total bitumen,химинвест нпф,0,0
200125,lanxess,химинвест нпф,0,0


Теперь сделаем транслитерацию на строках, содержащих русские слова:

In [68]:
%%time
train.loc[train.name_1.str.contains(russian) | train.name_2.str.contains(russian), 'name_1'] = train.loc[train.name_1.str.contains(russian) | train.name_2.str.contains(russian),
                                                                                                         'name_1'].apply(lambda x: translit(x, "ru", reversed=True))
train.loc[train.name_1.str.contains(russian) | train.name_2.str.contains(russian), 'name_2'] = train.loc[train.name_1.str.contains(russian) | train.name_2.str.contains(russian),
                                                                                                         'name_2'].apply(lambda x: translit(x, "ru", reversed=True))
test.loc[test.name_1.str.contains(russian) | test.name_2.str.contains(russian), 'name_1'] = test.loc[test.name_1.str.contains(russian) | test.name_2.str.contains(russian),
                                                                                                       'name_1'].apply(lambda x: translit(x, "ru", reversed=True))
test.loc[test.name_1.str.contains(russian) | test.name_2.str.contains(russian),'name_2'] = test.loc[test.name_1.str.contains(russian) | test.name_2.str.contains(russian),
                                                                                                      'name_2'].apply(lambda x: translit(x, "ru", reversed=True))

Wall time: 5.74 s


In [69]:
(train.name_1.str.contains(russian) | train.name_2.str.contains(russian)).sum()

0

In [70]:
train.loc[24723]

name_1          dorken
name_2          dorken
is_duplicate         1
label                0
Name: 24723, dtype: object

При транслитерации могли произойти неоднозначные замены, поправим их:

In [71]:
train['name_1'] = train['name_1'].str.replace('w', 'v', regex=True)
train['name_2'] = train['name_2'].str.replace('w', 'v', regex=True)
test['name_1'] = test['name_1'].str.replace('w', 'v', regex=True)
test['name_2'] = test['name_2'].str.replace('w', 'v', regex=True)

train['name_1'] = train['name_1'].str.replace('ks', 'x', regex=True)
train['name_2'] = train['name_2'].str.replace('ks', 'x', regex=True)
test['name_1'] = test['name_1'].str.replace('ks', 'x', regex=True)
test['name_2'] = test['name_2'].str.replace('ks', 'x', regex=True)

In [72]:
# убираем неодинарные пробелы и пробелы в начале и конце строк
clean_spaces()

In [73]:
train.sample(10)

,name_1,name_2,is_duplicate,label
pair_id,,,,
384287,shipco,stp,0,0
161271,amtrans,panalpina uruguay mundiales,0,0
33511,robertson,okamoto shenzhen,0,0
390109,haosheng vina,acs,0,0
367526,zaklad metalovy gemo,vainternational,0,0
447554,greenchem,topfils,0,0
50198,nimsetha,aspire,0,0
181390,chasse sohn inh radecke,oldcastle building,0,0
433114,caleres,sinfa cables,0,0


In [74]:
train.query('is_duplicate == 1').sample(10)

,name_1,name_2,is_duplicate,label
pair_id,,,,
231764,synthomer peachtree lenox bl bonded 7320 state...,synthomer srl,1,0
210107,rompetrol,rompetrol rafinare,1,0
444587,evonik cryo,evonik cyro mobile,1,0
144631,arlanxeo,arlanxeo,1,1
48161,arlanxeo usa,arlanxeo canada,1,0
461222,shenzhen comlink,shenzhen comlink,1,0
169771,vertikal' sport,vertikal' sport,1,1
338104,asfaltos chova,asfaltos chova,1,1
437848,bridgestone india,bridgestone brasil comenrcio,1,0


In [75]:
train.duplicated().sum()

96

In [76]:
train = train.drop_duplicates()

## Удаление названий стран

In [77]:
# есть имена стран на других языках
for df in [train, test]:
    for series_name in ['name_1', 'name_2']:
        df[series_name] = df[series_name].str.replace('brasil', 'brazil')
        df[series_name] = df[series_name].str.replace('czechy', 'czechia')
        df[series_name] = df[series_name].str.replace('polska', 'poland')
        df[series_name] = df[series_name].str.replace('mexic\S*', 'mexico', regex=True)
        df[series_name] = df[series_name].str.replace('deutschland', 'germany')
        df[series_name] = df[series_name].str.replace('turk', 'turkey')
        df[series_name] = df[series_name].str.replace('nederland', 'netherlands')

In [78]:
countries = [country.name.lower() for country in pycountry.countries] + ['usa?', 'uk', 'america\S*',
                                                                         'north', 'south',
                                                                         'europe', 'asia']

In [79]:
%%time
for country in tqdm(countries):
    country_reg = re.compile('(^|\s+)'+country+'(\s+|$)')
    for df in [train, test]:
        df['name_1'] = df['name_1'].str.replace(country_reg, ' ', regex=True)
        df['name_2'] = df['name_2'].str.replace(country_reg, ' ', regex=True)
clean_spaces()    

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [07:17<00:00,  1.71s/it]


Wall time: 7min 21s


### Убираем названия городов

Названия китайских провинций из википедии https://wikitravel.org/en/List_of_Chinese_provinces_and_regions:

In [80]:
ch_prov = ['Anhui', 'Fujian', 'Gansu', 'Guangdong', 'Guizhou','Hainan','Hebe',
           'Heilongjiang','Henan','Hubei','Hunan','Jiangsu','Jiangxiv','Jilin',
           'Liaoning','Qinghai','Shaanxi','Shanxi','Sichuan','Yunnan','Zhejiang',]
ch_prov = [i.lower() for i in ch_prov]

In [81]:
cities_list = ['shenzhen', 'shanghai', 'guangzhou', 'guangdong', 'huizhou', #эти названия есть в трейне
               'shenyang', 'dongguan', 'qingdao', 'shenzhen', 'zhongshan',
               'hangzhou', 'tianjin', 'zhuhai', 'xiamen', 'changshu', "ningbo", 'suzhou',
               'uchkurgan' ] + ch_prov + ['hindustan', 'khawaja', 'jindal']
print(len(cities_list))

42


In [82]:
train['name_1'].str.contains('shanghai').sum()

7627

In [83]:
%%time
for city in tqdm(cities_list):
    train['name_1'] = train['name_1'].str.replace(city, ' ')
    train['name_2'] = train['name_2'].str.replace(city, ' ')
    test['name_1'] = test['name_1'].str.replace(city, ' ')
    test['name_2'] = test['name_2'].str.replace(city, ' ')
clean_spaces()

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:56<00:00,  1.35s/it]


Wall time: 1min 1s


In [84]:
train['name_1'].str.contains('shanghai').sum()

0

In [85]:
#удалим все цифры (которые отдельным "словом")
for df in [train, test]:
    for col_name in ['name_1', 'name_2']:
        df[col_name] = df[col_name].str.replace('(^|\s+)\d+(\s+|$)', '', regex=True)

In [86]:
train.duplicated().sum()

9057

In [87]:
train = train.drop_duplicates()

In [88]:
train.to_csv('train_clean.csv', index=True)
test.to_csv('test_clean.csv', index=True)

### Проверка полных дубликатов 2

In [91]:
train['full_duplicate'] = train.apply(lambda x: set(x.name_1.split()) == set(x.name_2.split()), axis=1).astype('int')

In [92]:
train[(train.full_duplicate == 1) & ((train.is_duplicate == 0) | (train.label == 0))].shape

(321, 5)

In [93]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate == 1, 'name_1'] = train.loc[train.full_duplicate == 1, 'name_2'].values

In [94]:
# проверим полные дубликаты в тесте
test['full_duplicate'] = test.apply(lambda x: set(x.name_1.split()) == set(x.name_2.split()), axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate == 1, 'name_1'] = test.loc[test.full_duplicate == 1, 'name_2'].values

Отсечем от первого имени одно слово с конца:

In [95]:
train['full_duplicate_1'] = train.apply(del1_word_n1, axis=1).astype('int')

In [96]:
train[(train.full_duplicate_1 == 1) & ((train.is_duplicate == 0) | (train.label == 0))].shape

(370, 6)

In [97]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate_1 == 1, 'name_1'] = train.loc[train.full_duplicate_1 == 1, 'name_2'].values

In [98]:
# отметим такие же дубликаты в тесте
test['full_duplicate_1'] = test.apply(del1_word_n1, axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate_1 == 1, 'name_1'] = test.loc[test.full_duplicate_1 == 1, 'name_2'].values

In [99]:
test[test['full_duplicate_1'] == 1].shape

(251, 5)

То же самое, но от второго названия уберем одно слово:

In [100]:
train['full_duplicate_2'] = train.apply(del1_word_n2, axis=1).astype('int')

In [101]:
train[(train.full_duplicate_2 == 1) & ((train.is_duplicate == 0) | (train.label == 0))].shape

(69, 7)

In [102]:
# сделаем одинаковый порядок слов
train.loc[train.full_duplicate_2 == 1, 'name_2'] = train.loc[train.full_duplicate_2 == 1, 'name_1'].values

In [103]:
# удалим эти столбцы
train['label'] = train[['label', 'full_duplicate','full_duplicate_1','full_duplicate_2']].max(axis=1)
train = train.drop(['full_duplicate', 'full_duplicate_1', 'full_duplicate_2'], axis=1)

In [104]:
# отметим такие же дубликаты в тесте
test['full_duplicate_2'] = test.apply(del1_word_n2, axis=1).astype('int')
# сделаем одинаковый порядок слов
test.loc[test.full_duplicate_2 == 1, 'name_2'] = test.loc[test.full_duplicate_2 == 1, 'name_1'].values

In [105]:
# в тесте сделаем столбец разметки на основе сравнения
test['label'] = test[['label','full_duplicate','full_duplicate_1','full_duplicate_2']].max(axis=1)
test = test.drop(['full_duplicate', 'full_duplicate_1', 'full_duplicate_2'], axis=1)

In [106]:
train[(train.label == 1) & (train.is_duplicate == 0)].shape

(404, 4)

In [107]:
test[test.label==1].shape

(859, 3)

### Проверка дубликатов первого слова

In [108]:
def check_first_words(row):
    s1 = row['name_1']
    s2 = row['name_2']
    try:
        fw1 = s1.split()[0]
    except Exception:
        fw1 = s1
    try:
        fw2 = s2.split()[0]
    except Exception:
        fw2 = s2
    return fw1 == fw2

In [109]:
%%time
train['first_word_duplicate']  = train.apply(check_first_words, axis=1).astype("int")
test['first_word_duplicate']  = test.apply(check_first_words, axis=1).astype("int")

Wall time: 8.56 s


In [110]:
train.head()

,name_1,name_2,is_duplicate,label,first_word_duplicate
pair_id,,,,,
1,iko,enormous,0,0,0
2,apcotex,technocraft,0,0,0
3,rishichem distributors,dsa,0,0,0
4,povermax factory,one,0,0,0
5,tress,longyou park,0,0,0


In [111]:
train['label'] = train[['label', 'first_word_duplicate']].max(axis=1)
train = train.drop(['first_word_duplicate'], axis=1)

In [112]:
train[(train.label == 1) & (train.is_duplicate == 0)].shape

(2231, 4)

In [113]:
test['label'] = test[['label', 'first_word_duplicate']].max(axis=1)
test = test.drop(['first_word_duplicate'], axis=1)

In [114]:
test[test.label==1].shape

(2181, 3)

In [125]:
train["org_name_1"] = train_raw["name_1"]
train["org_name_2"] = train_raw["name_2"]

test["org_name_1"] = test_raw["name_1"]
test["org_name_2"] = test_raw["name_2"]

# Метрики сходства строк

### Расстояния

In [126]:
# textdistance
td_names = [
    
    'mlipns', 
    'hamming',
    'hamming_norm',
    'levenshtein',
    'levenshtein_norm',
    'damerau_levenshtein',
    'jaro_winkler',
    'strcmp95',
    'tanimoto',
    'monge_elkan',
    'lcsseq',
    'lcsstr',
     
    'needleman_wunsch',
    'needleman_wunsch_norm',
    'gotoh',
    'smith_waterman',
    'smith_waterman_norm',
    
    'ratcliff_obershelp',
   
    'cosine', 
    'jaccard',
    'sorensen'

]

td_methods = [
    td.mlipns.normalized_similarity,
    td.hamming.similarity,
    td.hamming.normalized_similarity,
    td.levenshtein.similarity,
    td.levenshtein.normalized_similarity,
    td.damerau_levenshtein.normalized_similarity,
    td.jaro_winkler.normalized_similarity,
    td.strcmp95.normalized_similarity,
    td.tanimoto.normalized_similarity,
    td.monge_elkan.normalized_similarity,
    td.lcsseq.normalized_similarity,
    td.lcsstr.normalized_similarity,
    
    td.needleman_wunsch.similarity,
    td.needleman_wunsch.normalized_similarity,
    td.gotoh.normalized_similarity,
    td.smith_waterman.normalized_similarity,
    
    td.ratcliff_obershelp.similarity,
    
    td.cosine.similarity,
    td.jaccard.similarity,
    td.sorensen.similarity
]

In [127]:
def td_distance_count(row, method, name):
    n1, n2 = row['name_1'], row['name_2']
    if n1 == "" and n2 == "":
        n1, n2 = row['org_name_1'], row['org_name_2']
    
    if name in ['jaccard', 'sorensen']:
        return method(n1.split(' '), n2.split(' '))
    
    return method(''.join(n1.split(' ')), ''.join(n2.split(' ')))

In [128]:
# fuzz
fuzz_names = [
    'ratio',
    'partial_ratio',
    'token_sort_ratio',
    'token_set_ratio'
]

fuzz_methods = [
    fuzz.ratio,
    fuzz.partial_ratio,
    fuzz.token_sort_ratio,
    fuzz.token_set_ratio
]

In [129]:
def fuzz_distance_count(row, method, name):
    n1, n2 = row['name_1'], row['name_2']
    if n1 == "" and n2 == "":
        n1, n2 = row['org_name_1'], row['org_name_2']
    
    return method(''.join(n1.split(' ')), ''.join(n2.split(' ')))/100

In [130]:
# ddf для ускорения apply
train_ddf = dd.from_pandas(train, npartitions=4)
test_ddf = dd.from_pandas(test, npartitions=2)

In [131]:
%%time
for name, method in zip(td_names, td_methods):
    print(name)
    train[name] = train_ddf.apply(td_distance_count, axis=1, args=(method, name), meta=(name, 'float32'))
    test[name] = test_ddf.apply(td_distance_count, axis=1, args=(method, name), meta=(name, 'float32'))

mlipns
[########################################] | 100% Completed | 18.3s
[########################################] | 100% Completed | 20.6s
[########################################] | 100% Completed | 21.0s
[########################################] | 100% Completed |  8.1s
[########################################] | 100% Completed |  8.0s
[########################################] | 100% Completed |  7.6s
hamming
[########################################] | 100% Completed | 16.3s
[########################################] | 100% Completed | 17.6s
[########################################] | 100% Completed | 16.1s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.5s
[########################################] | 100% Completed |  6.1s
hamming_norm
[########################################] | 100% Completed | 16.6s
[########################################] | 100% Completed | 15.8s
[###################

[########################################] | 100% Completed | 24.9s
[########################################] | 100% Completed | 24.8s
[########################################] | 100% Completed | 24.6s
[########################################] | 100% Completed |  8.8s
[########################################] | 100% Completed |  9.8s
[########################################] | 100% Completed |  9.6s
Wall time: 2h 53min 26s


In [132]:
%%time
for name, method in zip(fuzz_names, fuzz_methods):
    print(name)
    train[name] = train_ddf.apply(fuzz_distance_count, axis=1, args=(method, name), meta=(name, 'float32'))
    test[name] = test_ddf.apply(fuzz_distance_count, axis=1, args=(method, name), meta=(name, 'float32'))

ratio
[########################################] | 100% Completed | 13.2s
[########################################] | 100% Completed | 13.4s
[########################################] | 100% Completed | 13.2s
[########################################] | 100% Completed |  5.6s
[########################################] | 100% Completed |  4.6s
[########################################] | 100% Completed |  4.4s
partial_ratio
[########################################] | 100% Completed | 19.6s
[########################################] | 100% Completed | 20.9s
[########################################] | 100% Completed | 21.2s
[########################################] | 100% Completed |  7.0s
[########################################] | 100% Completed |  6.6s
[########################################] | 100% Completed |  6.9s
token_sort_ratio
[########################################] | 100% Completed | 19.1s
[########################################] | 100% Completed | 17.7s
[##########

In [133]:
train.to_csv('train_with_features.csv')
test.to_csv('test_with_features.csv')